In [1]:
!pip install -q -U bitsandbytes > /dev/null
!pip install -q -U git+https://github.com/huggingface/transformers.git > /dev/null
!pip install -q -U git+https://github.com/huggingface/peft.git > /dev/null
!pip install -q -U git+https://github.com/huggingface/accelerate.git > /dev/null
!pip install -q datasets > /dev/null

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType,PromptTuningInit, PromptTuningConfig
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import os


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

device = "cuda"
model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"

text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

In [4]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank",
                       "sentences_allagree")

dataset = dataset["train"].train_test_split(test_size=0.1)

dataset["validation"] = dataset["test"]

del dataset["test"]

classes = dataset["train"].features["label"].names

dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to /root/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'Featuring the S60 user interface , this 135-gram model also comes with a 2.4-inch quarter VGA display and 100 megabytes of internal memory , expandable through the Micro Secure Digital slot .',
 'label': 1,
 'text_label': 'neutral'}

In [5]:
classes

['negative', 'neutral', 'positive']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs,
                             max_length=max_length,
                             padding="max_length",
                             truncation=True,
                             return_tensors="pt")
    labels = tokenizer(targets,
                       max_length=2,
                       padding="max_length",
                       truncation=True,
                       return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

In [7]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [8]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(train_dataset,
                              shuffle=True,
                              collate_fn=default_data_collator,
                              batch_size=batch_size,
                              pin_memory=True)
eval_dataloader = DataLoader(eval_dataset,
                             collate_fn=default_data_collator,
                             batch_size=batch_size,
                             pin_memory=True)

In [9]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM,
                                 inference_mode=False,
                                 num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.13308583065659835


In [10]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
model.to(device)

In [21]:
train_dataset[0].items()

dict_items([('input_ids', [3, 18772, 8, 180, 3328, 1139, 3459, 3, 6, 48, 209, 2469, 18, 5096, 825, 92, 639, 28, 3, 9, 3, 17638, 18, 4976, 2893, 584, 6302, 1831, 11, 910, 13950, 17770, 7, 13, 3224, 2594, 3, 6, 4405, 179, 190, 8, 5893, 18865, 4190, 4918, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('attention_mask', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('labels', [7163, 1])])

In [22]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []

    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 29/29 [00:06<00:00,  4.26it/s]


epoch=0: train_ppl=tensor(4.7776, device='cuda:0') train_epoch_loss=tensor(1.5639, device='cuda:0') eval_ppl=tensor(1.1442, device='cuda:0') eval_epoch_loss=tensor(0.1347, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.24it/s]


epoch=1: train_ppl=tensor(1.1245, device='cuda:0') train_epoch_loss=tensor(0.1173, device='cuda:0') eval_ppl=tensor(1.0821, device='cuda:0') eval_epoch_loss=tensor(0.0789, device='cuda:0')


100%|██████████| 29/29 [00:07<00:00,  4.12it/s]


epoch=2: train_ppl=tensor(1.0786, device='cuda:0') train_epoch_loss=tensor(0.0757, device='cuda:0') eval_ppl=tensor(1.0687, device='cuda:0') eval_epoch_loss=tensor(0.0665, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.20it/s]


epoch=3: train_ppl=tensor(1.0730, device='cuda:0') train_epoch_loss=tensor(0.0704, device='cuda:0') eval_ppl=tensor(1.0585, device='cuda:0') eval_epoch_loss=tensor(0.0568, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.23it/s]

epoch=4: train_ppl=tensor(1.0730, device='cuda:0') train_epoch_loss=tensor(0.0705, device='cuda:0') eval_ppl=tensor(1.0555, device='cuda:0') eval_epoch_loss=tensor(0.0540, device='cuda:0')


In [23]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100

print(f"{accuracy=} % on the evaluation dataset")

print(f"{eval_preds[:10]=}")

print(f"{dataset['validation']['text_label'][:10]=}")


accuracy=97.79735682819384 % on the evaluation dataset
eval_preds[:10]=['positive', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive']
dataset['validation']['text_label'][:10]=['positive', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral']


In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
peft_model_id = "Kamaljp/t5-large_PREFIX_TUNING_SEQ2SEQ"

model.push_to_hub("Kamaljp/t5-large_PREFIX_TUNING_SEQ2SEQ", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kamaljp/t5-large_PREFIX_TUNING_SEQ2SEQ/commit/44b2f94991d5025a83b5ce9edbd68567c1616b7c', commit_message='Upload model', commit_description='', oid='44b2f94991d5025a83b5ce9edbd68567c1616b7c', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
from peft import PeftModel, PeftConfig

peft_model_id = "Kamaljp/t5-large_PREFIX_TUNING_SEQ2SEQ"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [27]:
inputs = tokenizer(
    "The Indian beer market made up 24.41 million liters in January , a rise of 18 percent.",
    return_tensors="pt",
)

In [28]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive']


['positive']

Working on Causal Language modelling

In [3]:
device = "cuda"
model_name_or_path = "bigscience/bloomz-560m"
tokenizer_name_or_path = "bigscience/bloomz-560m"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the tweet is a complaint or not:",
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "twitter_complaints"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

In [4]:
dataset = load_dataset("ought/raft", dataset_name)
dataset["train"][0]

Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

Dataset raft downloaded and prepared to /root/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}

In [5]:
dataset["train"].feature

Dataset({
    features: ['Tweet text', 'ID', 'Label'],
    num_rows: 50
})

In [7]:
dataset["train"].features["Label"].names

['Unlabeled', 'complaint', 'no complaint']

In [8]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
dataset["train"][0]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [11]:
tokenizer.eos_token_id

2

In [8]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])

print(target_max_length)


3


In [11]:
import torch

def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [13]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["train"]


train_dataloader = DataLoader(train_dataset, shuffle=True,
                              collate_fn=default_data_collator,
                              batch_size=batch_size,
                              pin_memory=True)
eval_dataloader = DataLoader(eval_dataset,
                             collate_fn=default_data_collator,
                             batch_size=batch_size,
                             pin_memory=True)

In [14]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

model = get_peft_model(model, peft_config)

print(model.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358
None


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [18]:
model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
peft_model_id = "Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"
model.push_to_hub("Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM/commit/7a7a82d99c176ec88a2ed91dbef9004e269ce9b5', commit_message='Upload model', commit_description='', oid='7a7a82d99c176ec88a2ed91dbef9004e269ce9b5', pr_url=None, pr_revision=None, pr_num=None)